<a href="https://colab.research.google.com/github/nirjana/Fuse_AI-Fellowship/blob/master/Movie_recommendation_using_semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install gdown
!pip install chromadb
!pip install -U sentence-transformers

In [2]:
!gdown 1C0Qn6FUwNQtq5MowDF3Eheg5ilE4pPIB

Downloading...
From: https://drive.google.com/uc?id=1C0Qn6FUwNQtq5MowDF3Eheg5ilE4pPIB
To: /content/movies.csv
100% 494k/494k [00:00<00:00, 6.90MB/s]


# New Section

In [3]:
import pandas as pd
df=pd.read_csv("/content/movies.csv")
df.dropna(inplace=True)

In [4]:
df.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [5]:
feature =df.title+ " " +df['genres'].apply(lambda x:x.replace("|",""))



In [6]:
feature =feature.str.lower()
feature

,0
0,toy story (1995) adventureanimationchildrencom...
1,jumanji (1995) adventurechildrenfantasy
2,grumpier old men (1995) comedyromance
3,waiting to exhale (1995) comedydramaromance
4,father of the bride part ii (1995) comedy
...,...
9737,black butler: book of the atlantic (2017) acti...
9738,no game no life: zero (2017) animationcomedyfa...
9739,flint (2017) drama
9740,bungo stray dogs: dead apple (2018) actionanim...


In [7]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = model.encode(feature.values).tolist()

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
embeddings[:3]

[[-0.05557648837566376,
  0.03857547417283058,
  0.06630285829305649,
  -0.024291271343827248,
  -0.0024993224069476128,
  0.027775680646300316,
  0.06635775417089462,
  -0.005739916581660509,
  -0.03150343522429466,
  0.039650045335292816,
  0.028574932366609573,
  0.019615614786744118,
  -0.02390487864613533,
  0.0745016410946846,
  0.029132138937711716,
  0.03808347135782242,
  0.042197708040475845,
  0.05751481279730797,
  0.021771950647234917,
  -0.06368257105350494,
  0.04517807066440582,
  -0.021053247153759003,
  0.07035157084465027,
  -0.007078646216541529,
  -0.060218583792448044,
  0.03032110631465912,
  -0.013608337379992008,
  -0.03633635863661766,
  -0.055747807025909424,
  0.006005003582686186,
  0.03822961077094078,
  0.01237956341356039,
  0.029585182666778564,
  -0.06360748410224915,
  0.008583533577620983,
  0.01771908439695835,
  -0.04399789124727249,
  -0.07327799499034882,
  0.009494212456047535,
  -0.017284899950027466,
  -0.026463469490408897,
  0.05252917855978

In [9]:
df.dtypes

,0
movieId,int64
title,object
genres,object


In [10]:
ids =df.movieId.astype(str).values.tolist()
metadata =df.to_dict('records')

In [11]:
import chromadb
client =chromadb.Client()

In [12]:
collection = client.create_collection("vector_db") # one time

In [13]:
collection.add(ids = ids,
               documents=feature.values.tolist(),
               embeddings = embeddings,
               metadatas=metadata)

In [14]:
user_feature =input("what kind of genres of movies do yu like?")

what kind of genres of movies do yu like?i would love to watch action and love story moview


In [15]:
user_feature

'i would love to watch action and love story moview'

In [16]:
recommendations =collection.query(query_texts =[user_feature],n_results = 3)

In [17]:
print(recommendations)

{'ids': [['60894', '600', '39183']], 'distances': [[1.0271834135055542, 1.05012047290802, 1.0638071298599243]], 'metadatas': [[{'genres': 'Drama|Romance|War', 'movieId': 60894, 'title': 'Edge of Love, The (2008)'}, {'genres': 'Action|Comedy|Crime', 'movieId': 600, 'title': 'Love and a .45 (1994)'}, {'genres': 'Drama|Romance', 'movieId': 39183, 'title': 'Brokeback Mountain (2005)'}]], 'embeddings': None, 'documents': [['edge of love, the (2008) dramaromancewar', 'love and a .45 (1994) actioncomedycrime', 'brokeback mountain (2005) dramaromance']], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}
